In [1]:
import pandas as pd
import numpy as np

In [2]:
def transform(file):
    r = pd.DataFrame()
    r['index'] = file['index']
    r['resultados_24'] = file['currentResults']

    d = {
        'territoryKey': file[0].territoryKey,
        'território': file[0].territoryFullName,
        'resultados': r
    }

    row = pd.DataFrame(data=d, index='territoryKey')

    return row

In [3]:
local = pd.read_csv('datasets/territory_codes_local.csv')
foreign = pd.read_csv('datasets/territory_codes_foreign.csv')

codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
)

codes

,territoryKey,freguesia,concelho,distrito
0,LOCAL-430101,Altares,Angra do Heroísmo,Açores
1,LOCAL-430102,Angra (Nossa Senhora da Conceição),Angra do Heroísmo,Açores
2,LOCAL-430103,Angra (Santa Luzia),Angra do Heroísmo,Açores
3,LOCAL-430104,Angra (São Pedro),Angra do Heroísmo,Açores
4,LOCAL-430105,Angra (Sé),Angra do Heroísmo,Açores
...,...,...,...,...
19,FOREIGN-930299,Postos Consulares da China,China,Fora da Europa
20,FOREIGN-920799,Postos Consulares dos Estados Unidos da América,Estados Unidos da América,Fora da Europa
21,FOREIGN-919999,Postos Consulares dos países de África,Países de África,Fora da Europa
22,FOREIGN-929999,Restantes Postos Consulares Dos Países da América,Restantes Países da América,Fora da Europa


In [4]:
t = pd.DataFrame(columns=['territoryKey', 'território', 'resultados'])
t['territoryKey'] = codes['territoryKey']

file = (
    pd.read_csv('datasets\Local\Açores\Angra do Heroísmo\Altares.csv')
)

file


,index,currentResults,displayMessage,foreignCounterMessage,nationalCounterMessage,previousResults,refreshTimeout,territoryFullName,territoryKey,territoryMessage,territoryName,time,timeOnServer,totalConsulates,totalConsulatesApproved,totalParishes,totalParishesApproved,warningMessage
0,availableMandates,NaN,NaN,NaN,NaN,NaN,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
1,blankVotes,13,NaN,NaN,NaN,6,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
2,blankVotesPercentage,2.8200000000000003,NaN,NaN,NaN,1.6800000000000002,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
3,compensation,False,NaN,NaN,NaN,False,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
4,displayMessage,NaN,NaN,NaN,NaN,NaN,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
5,hasNoVoting,False,NaN,NaN,NaN,NaN,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
6,nullVotes,3,NaN,NaN,NaN,2,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
7,nullVotesPercentage,0.65,NaN,NaN,NaN,0.56,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
8,numberParishes,1,NaN,NaN,NaN,NaN,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN
9,numberVoters,461,NaN,NaN,NaN,NaN,20,Altares,LOCAL-430101,NaN,Altares,16:57,0,24,24,3092,3092,NaN


In [13]:
r = pd.DataFrame()
r['index'] = file['index']
r.set_index('index', inplace=True)

n = pd.Series(file['currentResults'].drop(11))


In [ ]:
# results = pd.DataFrame(columns=['territoryKey', 'território', 'resultados'])

# for i, row in codes.iterrows():
#     tkey = row['territoryKey']
#     loc = tkey.split('-')[0].capitalize()
#     dist = row['distrito']
#     county = row['concelho']
#     parish = row['freguesia']

#     path = 'datasets/' + loc + '/' + dist + '/' + county + '/' + parish + '.csv'

#     file = pd.read_csv(path)
#     row = transform(file)

#     results.loc[len(results.index)] = row

# results